# Project 5: Wrangle and Analyze Data

## Prepared by: Lamia Alshawi

## Gather

In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import os 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json

In [2]:
# gather and read WeRateDogs Twitter archive CSV
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# downloaded the image prediction file programmatically using the Requests library and save tsv to file
url = ('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
response = requests.get(url)
with open('image-predictions.tsv', mode ='wb') as file:
    file.write(response.content)
# read file
image_predictions_df = pd.read_csv('image-predictions.tsv', sep='\t')

In [4]:
# extract the file from the zipfile
with open('tweet-json.zip','rb') as f:
    tweets = zipfile.ZipFile(f)
    tweets.extractall()

In [10]:
# read the file in DataFrame
with open('tweet-json copy', 'r') as f:
    tweet_json_df = pd.read_json(f, lines= True, encoding = 'utf-8')

# select the columns of interest 'id', 'favorite_count','retweet_count'
tweet_json_df = tweet_json_df.loc[:,['id','favorite_count','retweet_count']]

## Assess

Now we assess each dataframe for quality and tidiness and describe each column variable. Assess them visually and programmatically for quality and tidiness issues.

In [ ]:
twitter_archive_df

In [ ]:
image_predictions_df

In [ ]:
tweet_json_df

In [ ]:
# get Twitter Archive info
twitter_archive_df.info()

In [ ]:
# Twitter Archive description
twitter_archive_df.describe()

In [ ]:
# Twitter Archive sample
twitter_archive_df.sample(5)

In [ ]:
# image predictions info
image_predictions_df.info()

In [ ]:
# image predictions description
image_predictions_df.describe()

In [ ]:
# image predictions sample
image_predictions_df.sample(5)

In [ ]:
# tweet_json data info
tweet_json_df.info()

In [ ]:
# tweet_json data description
tweet_json_df.describe()

In [ ]:
# tweet_json data sample
tweet_json_df.sample(5)

Quality

- Remove data contains retweets because we are only interested in original tweets
- Change tweet_id to a string/object instead of integer
- drop unnecessary columns that are not needed for analysis
- in the name column there are values that are inacuartely named like "a", "an", "all"
- the name column also has none instead of NaN 
- timestamp is an object we need to change it into datetime
- Some tweets have the same jpg_url
- some tweets have no image
- not need the tweets beyond August 1st, 2017.

Tidiness
- Merge all data frames by tweet_id.
- Combine dog stages doggo, puppo, pupper, floofer into one column 'dog_stage'

## Clean

In [ ]:
tweet_json_clean = tweet_json_df.copy()
twitter_archive_clean = twitter_archive_df.copy()
image_predictions_clean= image_predictions_df.copy()

### Tidiness

##### Define

Merge all data frames by tweet_id.

##### Code

In [ ]:
twitter_archive_clean = pd.merge(left=twitter_archive_clean, right=tweet_json_clean, left_on='tweet_id', right_on='id', how='inner')

In [ ]:
twitter_df = twitter_archive_clean.merge(image_predictions_clean, on='tweet_id', how='inner')

##### Test

In [ ]:
twitter_df.info()

#### Define

Combine dog stage columns (doggo, floofer, pupper, puppo) into one 'dog_stage'

#### Code

In [ ]:
# Extract dog stages from 'text' column into new 'dog_stage' column 
twitter_df['dog_stage'] = twitter_df['text'].str.extract('(doggo|floofer|pupper|puppo)')

In [ ]:
twitter_df[['dog_stage','doggo', 'floofer', 'pupper', 'puppo']].head(15)

In [ ]:
# now drop doggo, floofer, pupper, and puppo columns
twitter_df = twitter_df.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

#### Test

In [ ]:
twitter_df.dog_stage.value_counts()

### Quality

#### Define

Remove data contains retweets because we are only interested in original tweets

#### Code

In [ ]:
twitter_df = twitter_df.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

#### Test

In [ ]:
twitter_df.info()

#### Define

Change tweet_id to a string/object instead of integer

#### Code

In [ ]:
twitter_df['tweet_id'] = twitter_df['tweet_id'].astype(str)

#### Test

In [ ]:
twitter_df.info()

#### Define

drop unnecessary columns that are not needed for analysis

#### Code

In [ ]:
twitter_df = twitter_df.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'img_num',  'source'], axis=1)

#### Test

In [ ]:
twitter_df.info()

#### Define

- name column there are values that are inacuartely named like "a", "an", "all"
- the name column also has none instead of NaN 

#### Code

In [ ]:
# Find all names are only lowercase letters
lowercase = []
for row in twitter_df['name']:
    if row[0].islower() and row not in lowercase:
        lowercase.append(row)
print(lowercase)

In [ ]:
# replace lowercase letters with a NaN
twitter_df['name'].replace(lowercase, np.nan, inplace = True)

In [ ]:
# replace None with a NaN
twitter_df['name'].replace('None', np.nan, inplace = True)

#### Test

In [ ]:
twitter_df['name'].value_counts()

#### Define

timestamp is an object we need to change it into datetime

#### Code

In [ ]:
twitter_df['timestamp'] = pd.to_datetime(twitter_df.timestamp)

#### Test

In [ ]:
twitter_df.info()

#### Define

Some tweets have the same jpg_url

#### Code

In [ ]:
# how many duplicated images 
sum(twitter_df.jpg_url.duplicated())

In [ ]:
# drop duplicated images
twitter_df = twitter_df.drop_duplicates(subset=['jpg_url'], keep='last')

In [ ]:
# drop tweets with no images
twitter_df = twitter_df.dropna(subset = ['jpg_url'])

#### Test

In [ ]:
twitter_df.info()

#### Define

we do not need the tweets beyond August 1st, 2017.

#### Code

In [ ]:
tweets_august = '2017-08-01 00:00:00'
twitter_df = twitter_df[(twitter_df['timestamp'] < tweets_august)]

#### Test

In [ ]:
(twitter_df['timestamp'] > tweets_august).value_counts()

## Storing, Analyzing, and Visualizing Data

In [ ]:
twitter_df.to_csv('twitter_archive_master.csv', index = False)

In [ ]:
dognames = twitter_df['name'].value_counts().head(10)
dognames.plot(kind='barh', title = 'Common Dog Names');

In [ ]:
sns.regplot(x="retweet_count", y="favorite_count", data=twitter_df, scatter_kws={'alpha':0.2})
plt.title('Retweet vs Favorite', size=16)
plt.xlabel('Retweets', size=12)
plt.ylabel('Favorites', size=12)

In [ ]:
dogbreeds = twitter_df.p1.value_counts()[0:5]
dogbreeds.plot(kind = 'pie', autopct='%1.0f%%')
plt.title('Most Predicted Dog Breeds', size=16)

In [ ]:
dogstage = twitter_df.dog_stage.value_counts()
dogstage.plot(kind = 'pie', autopct='%1.0f%%')
plt.title('Most Common Dog Stages', size=16)